In [18]:
from pathlib import Path
from time import sleep

from lumigator_schemas.datasets import DatasetFormat
from lumigator_schemas.workflows import WorkflowCreateRequest, WorkflowStatus
from lumigator_sdk.lumigator import LumigatorClient
from lumigator_sdk.strict_schemas import ExperimentIdCreate


def wait_for_workflow_complete(lumi_client_int: LumigatorClient, workflow_id: str):
    """Wait for a workflow to complete."""
    workflow_details = lumi_client_int.workflows.get_workflow(workflow_id)
    while workflow_details.status not in [WorkflowStatus.SUCCEEDED, WorkflowStatus.FAILED]:
        sleep(5)
        workflow_details = lumi_client_int.workflows.get_workflow(workflow_id)
    return workflow_details


LUMI_HOST = "localhost:8000"
with Path.open("../lumigator/sample_data/dialogsum_exc.csv") as file:
    dialog_data = file.read()

In [ ]:
client = LumigatorClient(api_host=LUMI_HOST)
dataset_response = client.datasets.create_dataset(dataset=dialog_data, format=DatasetFormat.JOB)
dataset_id = dataset_response.id
dataset_id

In [ ]:
# Create an experiment
request = ExperimentIdCreate(
    name="Basic Test",
    description="Let's compare a few models on my dialogsum dataset!",
)
experiment_response = client.experiments.create_experiment(request)
experiment_id = experiment_response.id
experiment_id

In [21]:
# Let's run the Deepseek R1 https://api-docs.deepseek.com/quick_start/pricing
request = WorkflowCreateRequest(
    name="Deepseek R1",
    description="Test workflow for inf and eval",
    model="deepseek/deepseek-reasoner",
    model_url="deepseek/deepseek-reasoner",
    dataset=dataset_id,
    experiment_id=experiment_id,
    max_samples=1,
)
workflow_1_response = client.workflows.create_workflow(request)
workflow_1_id = workflow_1_response.id

In [ ]:
# Wait till the workflow is done
workflow_1_details = wait_for_workflow_complete(client, workflow_1_id)
# pretty print the output as json
workflow_1_details.model_dump()

In [ ]:
# Get the results of the experiment
experiment_results = client.experiments.get_experiment(experiment_id)
experiment_results.model_dump()